# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_df = pd.read_csv("../output_data/cities.csv")
# Rename the column with unnamed description
city_df = city_df.rename(columns={"Unnamed: 0":"City_ID"})
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Viedma,-40.8135,-62.9967,70.32,73,71,17.38,AR,1614690579
1,1,Port Moresby,-9.4431,147.1797,75.20,94,40,5.75,PG,1614690409
2,2,Bassar,9.2500,0.7833,96.80,16,0,11.50,TG,1614690579
3,3,Chuzhou,32.3219,118.2978,46.00,49,26,1.99,CN,1614690579
4,4,Makkah al Mukarramah,21.4267,39.8261,84.38,29,9,8.90,SA,1614690460


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# Heatmap of humidity
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]


fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
# Create layer that reveals humidity levels in the chosen locations
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# Update our data frame as we apply each criteria of the ideal weather
ideal_temp = city_df.loc[(city_df['Max Temp'] < 80) & (city_df['Max Temp'] > 65)]
ideal_wind = ideal_temp[ideal_temp['Wind Speed'] < 10]
ideal_cloud = ideal_wind[ideal_wind['Cloudiness'] == 0]
ideal_cities = ideal_cloud.dropna()

ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Hermanus,-34.4187,19.2345,77.00,61,0,5.01,ZA,1614690553
13,13,Mapastepec,15.4333,-92.9000,74.34,61,0,3.51,MX,1614690581
102,102,Barra de Tecoanapa,16.5167,-98.7500,70.93,85,0,2.73,MX,1614690595
180,180,Chāmpua,22.0833,85.6667,78.93,17,0,3.22,IN,1614690607
204,204,Nova Prata,-28.7839,-51.6100,78.01,52,0,1.99,BR,1614690420


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Create a data frame to store hotel name, city, country, lat and lng
hotel_df = ideal_cities[['City','Country', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Hermanus,ZA,-34.4187,19.2345,
13,Mapastepec,MX,15.4333,-92.9000,
102,Barra de Tecoanapa,MX,16.5167,-98.7500,
180,Chāmpua,IN,22.0833,85.6667,
204,Nova Prata,BR,-28.7839,-51.6100,
208,Bela,IN,25.9333,81.9833,
224,Taltal,CL,-25.4000,-70.4833,
302,Constantine,DZ,36.3650,6.6147,
366,Antofagasta,CL,-23.6500,-70.4000,
377,Prachuap Khiri Khan,TH,12.2167,99.7500,


In [51]:
import json
# Reset the DataFrame index values
hotel_df = hotel_df.reset_index(drop=True)
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

hotels = []

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_response = requests.get(base_url, params=params)
    print(hotel_response.url)

    # convert to json 
    hotel_data = hotel_response.json()
    print(json.dumps(hotel_data, indent = 4))

    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    
    except Exception:
        print("Not Found... skipping.")
hotel_df

7.9573323,
                    "lng": 75.777846
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.9586812802915,
                        "lng": 75.7791949802915
                    },
                    "southwest": {
                        "lat": 27.9559833197085,
                        "lng": 75.7764970197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Mountain Peak",
            "photos": [
                {
                    "height": 1800,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105036137283268468638\">Mohit Choudhary</a>"
                    ],
                    "photo_reference": "ATtYBwLFGfupMpMJXg-zRD2lizw3AqbI4WlrMgis6yk7YvqoVfC6R1ntT8ayMmQ2uD5PXAN9VOq9DZBru2GF_H5RTpvUEUj3udzDZ-y5mCJ7QwnbZBKePySZhis6S

,City,Country,Lat,Lng,Hotel Name
0,Hermanus,ZA,-34.4187,19.2345,Misty Waves Boutique Hotel
1,Mapastepec,MX,15.4333,-92.9000,Hotel San Carlos
2,Barra de Tecoanapa,MX,16.5167,-98.7500,Meson Casa de Piedra
3,Chāmpua,IN,22.0833,85.6667,ANSARI COTTAGE
4,Nova Prata,BR,-28.7839,-51.6100,Condall Palace Hotel
5,Bela,IN,25.9333,81.9833,
6,Taltal,CL,-25.4000,-70.4833,Residencial Karime
7,Constantine,DZ,36.3650,6.6147,
8,Antofagasta,CL,-23.6500,-70.4000,
9,Prachuap Khiri Khan,TH,12.2167,99.7500,


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))